In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import statsmodels.formula.api as smf 
import statsmodels.api as sm 
from sklearn.model_selection import train_test_split 
import itertools
%matplotlib inline
import numpy as np
from scipy import stats
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR, LinearSVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_absolute_error #MAE
from sklearn.metrics import mean_squared_error #MSE
from sklearn.metrics import make_scorer, r2_score
import xgboost as xgb
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [2]:
def rmse(y_true,y_pred):
    rmse = np.sqrt(mean_squared_error(y_true,y_pred))
    print('rmse',rmse)
    return rmse

In [3]:
df_clean = pd.read_csv('../01_Outputs/df_clean.csv')

In [4]:
category_table = pd.read_csv('../01_Outputs/Segment_CategoryTable.csv')

In [5]:
df_clean = pd.merge(df_clean, category_table, on='Segment_Name', how='left')

In [6]:
df_clean['SPEC_length_x_width'] = df_clean['SPEC_overall_length_mm'] * df_clean['SPEC_overall_width_mm']
df_clean['SPEC_MYMakeModel'] = df_clean['PROFILE_Model_year'].astype(str) + '_' + df_clean['PROFILE_Make'] + '_' + df_clean['PROFILE_Model']
df_clean = df_clean[~(df_clean['Segment_Name']=='SPORTS/PERFORMANCE')]
df_clean = df_clean[df_clean['SPEC_Enginetype_BEV']!=1]

In [7]:
df_modelgen = pd.read_csv('ModelChangeTiming.csv')

In [8]:
df_modelgen['SPEC_MYMakeModel'] = df_modelgen['PROFILE_Model_year'].astype(str) + '_' + df_modelgen['PROFILE_Make'] + '_' + df_modelgen['PROFILE_Model']


In [9]:
df_pop = df_clean[['PROFILE_Make', 'PROFILE_Model', 'PROFILE_Model_year', 'PROFILE_Version_name','TARGET_Volume', 'TARGET_Weighted_RetailPrice','PROFILE_Body_type', 'CATEGORY']]

In [10]:
df_pop['Highest_vol'] = df_pop.groupby(['PROFILE_Make', 'PROFILE_Model','PROFILE_Body_type', 'PROFILE_Model_year'])['TARGET_Volume'].transform('max')
df_pop['MostPopGrade'] = df_pop['Highest_vol'] == df_pop['TARGET_Volume']
df_pop['SPEC_MYMakeModel'] = df_pop['PROFILE_Model_year'].astype(str) + '_' + df_pop['PROFILE_Make'] + '_' + df_pop['PROFILE_Model']

In [11]:
df_pop = pd.merge(df_pop, df_modelgen[['SPEC_MYMakeModel', 'SPEC_Model generation', 'FullModelChange']], on='SPEC_MYMakeModel', how='left')

In [12]:
df_pop['SPEC_MYMakeModel'] = df_pop['PROFILE_Model_year'].astype(str) + '_' + df_pop['PROFILE_Make'] + '_' + df_pop['PROFILE_Model'] + '_' + df_pop['PROFILE_Body_type'] 

## Weighted MSRP Pattern(Same MSRP growing rate for all grades)

In [13]:
df_temp = df_pop[['SPEC_MYMakeModel','PROFILE_Model_year','PROFILE_Make','PROFILE_Model','PROFILE_Body_type','TARGET_Volume','TARGET_Weighted_RetailPrice', 'SPEC_Model generation', 'FullModelChange']]
df_temp['Volume_x_price'] = df_temp['TARGET_Volume'] * df_temp['TARGET_Weighted_RetailPrice']
df_temp['Volume_byMYModel'] = df_temp.groupby('SPEC_MYMakeModel')['TARGET_Volume'].transform('sum')
df_temp['Volume_index'] = df_temp['Volume_x_price'] / df_temp['Volume_byMYModel']
df_temp['MSRP_WeightedByVol'] = df_temp.groupby('SPEC_MYMakeModel')['Volume_index'].transform('sum')
df_featureeng = df_temp.groupby(['PROFILE_Model_year',\
                                 'PROFILE_Make','PROFILE_Model','SPEC_MYMakeModel','SPEC_Model generation', 'FullModelChange','PROFILE_Body_type'])\
                                 [['MSRP_WeightedByVol']].mean().reset_index()

In [14]:
df_featureeng['MY_max'] = df_featureeng.groupby(['PROFILE_Make','PROFILE_Model','PROFILE_Body_type' ,'SPEC_Model generation'])['PROFILE_Model_year'].transform('max')
df_featureeng['MY_min'] = df_featureeng.groupby(['PROFILE_Make','PROFILE_Model','PROFILE_Body_type' ,'SPEC_Model generation'])['PROFILE_Model_year'].transform('min')

In [15]:
df_featureeng = df_featureeng.sort_values(by=['PROFILE_Make', 'PROFILE_Model', 'PROFILE_Body_type', 'PROFILE_Model_year'], ascending=[True, True, True, True])

In [16]:
df_featureeng['MY_max_price'] = df_featureeng.apply(lambda x: x['MSRP_WeightedByVol'] if x['MY_max'] == x['PROFILE_Model_year'] else 0, axis=1)
df_featureeng['MY_min_price'] = df_featureeng.apply(lambda x: x['MSRP_WeightedByVol'] if x['MY_min'] == x['PROFILE_Model_year'] else 0, axis=1)

In [17]:
df_featureeng['MY_max_price'] = df_featureeng.groupby(['PROFILE_Make','PROFILE_Model', 'SPEC_Model generation'])['MY_max_price'].transform('max')
df_featureeng['MY_min_price'] = df_featureeng.groupby(['PROFILE_Make','PROFILE_Model', 'SPEC_Model generation'])['MY_min_price'].transform('max')

In [18]:
df_featureeng['Years_Modelchange'] = df_featureeng['MY_max'] - df_featureeng['MY_min']

In [19]:
df_featureeng['pricingup_rate'] = ((df_featureeng['MY_max_price'] - df_featureeng['MY_min_price'])/df_featureeng['MY_min_price'])/df_featureeng['Years_Modelchange']

In [20]:
df_featureeng = df_featureeng[df_featureeng['Years_Modelchange'] >2]

In [21]:
df_weighted = pd.merge(df_pop, df_featureeng[['SPEC_MYMakeModel', 'pricingup_rate']], on='SPEC_MYMakeModel', how='left')

In [22]:
len(df_weighted)

12693

In [23]:
df_weighted['Forecasted_MSRP'] = 0
for i in range(len(df_weighted)):
    if df_weighted['FullModelChange'][i] == 1:
        try:
            Make = df_weighted['PROFILE_Make'][i]
            Model = df_weighted['PROFILE_Model'][i]
            ModelYear = df_weighted['PROFILE_Model_year'][i]
            Version = df_weighted['PROFILE_Version_name'][i]
            df_weighted['Forecasted_MSRP'][i] = df_weighted[(df_weighted['PROFILE_Make']==Make)&(df_weighted['PROFILE_Model']==Model)&\
                    (df_weighted['PROFILE_Version_name']==Version)&(df_weighted['PROFILE_Model_year']==ModelYear-1)]['TARGET_Weighted_RetailPrice'] *\
                (1 + df_weighted[(df_weighted['PROFILE_Make']==Make)&(df_weighted['PROFILE_Model']==Model)&\
                    (df_weighted['PROFILE_Version_name']==Version)&(df_weighted['PROFILE_Model_year']==ModelYear-1)]['pricingup_rate'])
        except:
            pass


### Not possible to forecast because of the missing data point / Inconsistent grade structures

In [24]:
len(df_weighted[(df_weighted['FullModelChange']==1)&(df_weighted['Forecasted_MSRP']==0)]) / len(df_weighted[df_weighted['FullModelChange']==1])

0.6682738669238187

### Calculate RMSE

In [25]:
df_weighted_MidPrice_Midsize = df_weighted[df_weighted['CATEGORY'] =='MidPrice_Midsize']
df_weighted_LowPrice_Smallsize = df_weighted[df_weighted['CATEGORY'] =='LowPrice_Smallsize']
df_weighted_MidPrice_Largesize = df_weighted[df_weighted['CATEGORY'] =='MidPrice_Largesize']

In [26]:
df_weighted_MidPrice_Midsize = df_weighted_MidPrice_Midsize[df_weighted_MidPrice_Midsize['Forecasted_MSRP']>0]
rmse(df_weighted_MidPrice_Midsize['Forecasted_MSRP'], df_weighted_MidPrice_Midsize['TARGET_Weighted_RetailPrice'])

rmse 4883.14353004708


4883.14353004708

In [27]:
df_weighted_LowPrice_Smallsize = df_weighted_LowPrice_Smallsize[df_weighted_LowPrice_Smallsize['Forecasted_MSRP']>0]
rmse(df_weighted_LowPrice_Smallsize['Forecasted_MSRP'], df_weighted_LowPrice_Smallsize['TARGET_Weighted_RetailPrice'])

rmse 1569.208921947637


1569.208921947637

In [28]:
df_weighted_MidPrice_Largesize = df_weighted_MidPrice_Largesize[df_weighted_MidPrice_Largesize['Forecasted_MSRP']>0]
rmse(df_weighted_MidPrice_Largesize['Forecasted_MSRP'], df_weighted_MidPrice_Largesize['TARGET_Weighted_RetailPrice'])


rmse 1207.0634290842845


1207.0634290842845

# Using Past model's evaluation data

### Prepare the same data set as ML modeling

In [29]:
df_temp = df_clean[['SPEC_MYMakeModel','PROFILE_Model_year','PROFILE_Make','PROFILE_Model','TARGET_Volume','TARGET_Weighted_RetailPrice', 'RV_12mo','RV_36mo', 'RV_60mo']]
df_temp['Volume_x_price'] = df_temp['TARGET_Volume'] * df_temp['TARGET_Weighted_RetailPrice']
df_temp['Volume_byMYModel'] = df_temp.groupby('SPEC_MYMakeModel')['TARGET_Volume'].transform('sum')
df_temp['Volume_index'] = df_temp['Volume_x_price'] / df_temp['Volume_byMYModel']
df_temp['MSRP_WeightedByVol'] = df_temp.groupby('SPEC_MYMakeModel')['Volume_index'].transform('sum')

df_temp['Volume_x_RV_12mo'] = df_temp['TARGET_Volume'] * df_temp['RV_12mo']
df_temp['Volume_index'] = df_temp['Volume_x_RV_12mo'] / df_temp['Volume_byMYModel']
df_temp['RV_12mo_WeightedByVol'] = df_temp.groupby('SPEC_MYMakeModel')['Volume_index'].transform('sum')

df_temp['Volume_x_RV_36mo'] = df_temp['TARGET_Volume'] * df_temp['RV_36mo']
df_temp['Volume_index'] = df_temp['Volume_x_RV_36mo'] / df_temp['Volume_byMYModel']
df_temp['RV_36mo_WeightedByVol'] = df_temp.groupby('SPEC_MYMakeModel')['Volume_index'].transform('sum')

df_temp['Volume_x_RV_60mo'] = df_temp['TARGET_Volume'] * df_temp['RV_60mo']
df_temp['Volume_index'] = df_temp['Volume_x_RV_60mo'] / df_temp['Volume_byMYModel']
df_temp['RV_60mo_WeightedByVol'] = df_temp.groupby('SPEC_MYMakeModel')['Volume_index'].transform('sum')

In [30]:
df_featureeng = df_temp.groupby(['PROFILE_Model_year',\
                                 'PROFILE_Make','PROFILE_Model','SPEC_MYMakeModel'])\
[['MSRP_WeightedByVol', 'RV_12mo_WeightedByVol', 'RV_36mo_WeightedByVol', 'RV_60mo_WeightedByVol']].mean().reset_index()

In [31]:
df_featureeng = df_featureeng.sort_values(by=['PROFILE_Model', 'PROFILE_Model_year'], ascending=[1,1]).reset_index()

In [32]:
grouped = df_featureeng.groupby('PROFILE_Model')
df_featureeng['MSRP_prevMY_WeightedByVol'] = grouped['MSRP_WeightedByVol'].shift(1)
df_featureeng['MSRP_prevMY_WeightedByVol'] = \
    df_featureeng.apply(lambda x: x['MSRP_WeightedByVol'] \
                        if pd.isna(x['MSRP_prevMY_WeightedByVol']) else x['MSRP_prevMY_WeightedByVol'],axis=1)

In [33]:
df_featureeng['RV_12mo_prevMY_WeightedByVol'] = grouped['RV_12mo_WeightedByVol'].shift(1)
df_featureeng['RV_12mo_prevMY_WeightedByVol'] = \
    df_featureeng.apply(lambda x: x['RV_12mo_WeightedByVol'] \
                        if pd.isna(x['RV_12mo_prevMY_WeightedByVol']) else x['RV_12mo_prevMY_WeightedByVol'],axis=1)

In [34]:
df_featureeng['RV_36mo_prevMY_WeightedByVol'] = grouped['RV_36mo_WeightedByVol'].shift(1)
df_featureeng['RV_36mo_prevMY_WeightedByVol'] = \
    df_featureeng.apply(lambda x: x['RV_36mo_WeightedByVol'] \
                        if pd.isna(x['RV_36mo_prevMY_WeightedByVol']) else x['RV_36mo_prevMY_WeightedByVol'],axis=1)

In [35]:
df_featureeng['RV_60mo_prevMY_WeightedByVol'] = grouped['RV_60mo_WeightedByVol'].shift(1)
df_featureeng['RV_60mo_prevMY_WeightedByVol'] = \
    df_featureeng.apply(lambda x: x['RV_60mo_WeightedByVol'] \
                        if pd.isna(x['RV_60mo_prevMY_WeightedByVol']) else x['RV_60mo_prevMY_WeightedByVol'],axis=1)

In [36]:
df_clean = pd.merge(df_clean, df_featureeng[['SPEC_MYMakeModel', 'MSRP_prevMY_WeightedByVol',
                                  'RV_12mo_prevMY_WeightedByVol']], on='SPEC_MYMakeModel', how='left')

# Give the 2 years before values as assumption to forecast future MSRP

In [37]:
df_weighted['SPEC_MYMakeModelVersion'] = df_weighted['PROFILE_Model_year'].astype(str) + '_' + df_weighted['PROFILE_Make'] + '_' + df_weighted['PROFILE_Model']+'_'+ df_weighted['PROFILE_Body_type']  + '_' + df_weighted['PROFILE_Version_name']
df_clean['SPEC_MYMakeModelVersion'] = df_clean['PROFILE_Model_year'].astype(str) + '_' + df_clean['PROFILE_Make'] + '_' + df_clean['PROFILE_Model']+'_'+ df_clean['PROFILE_Body_type']  + '_' + df_clean['PROFILE_Version_name']

In [38]:
forecasted_models = df_weighted[df_weighted['Forecasted_MSRP']>0]['SPEC_MYMakeModelVersion'].tolist()

In [39]:
df_clean['FLAG_Forecasted'] = df_clean['SPEC_MYMakeModelVersion'].apply(lambda x: 1 if x in forecasted_models else 0)

In [40]:
df_clean = pd.concat([df_clean, df_weighted[['SPEC_Model generation']]],axis=1)

In [41]:
df_clean['SPEC_MYMakeModel_gen'] = df_clean['SPEC_MYMakeModel'] + '_' + df_clean['SPEC_Model generation'].astype(str)

In [42]:
backup = df_clean.copy()

In [64]:
df_clean = backup.copy()

In [65]:
prevyear_items = ['Edmunds','NCBS', 'OAO','RV', 'Inventory']
items_dct = {}
for item_large in prevyear_items:
    items_dct[item_large] = []
    for item_small in df_clean.filter(like=item_large).columns:
        df_clean['MY_Average_' + item_small] = df_clean.groupby(['SPEC_MYMakeModel_gen'])[item_small].transform('mean')
        items_dct[item_large].append(item_small)
for i in range(len(df_clean)):
    if df_clean['FLAG_Forecasted'][i] == True:
        print(i)
        Make = df_clean['PROFILE_Make'][i]
        Model = df_clean['PROFILE_Model'][i]
        ModelYear = df_clean['PROFILE_Model_year'][i]
        Version = df_clean['PROFILE_Version_name'][i]
        print(df_clean['SPEC_MYMakeModel_gen'][i])
        try:
            df_clean['MSRP_prevMY_WeightedByVol'][i] = df_clean[(df_clean['PROFILE_Make']==Make)&(df_clean['PROFILE_Model']==Model)&\
                    (df_clean['PROFILE_Model_year']==ModelYear-1)]['MSRP_prevMY_WeightedByVol'].reset_index(drop=True)[0]

            for item_large in prevyear_items:
                for item_small in items_dct[item_large]:
                    df_clean[item_small][i] = \
                    df_clean[(df_clean['PROFILE_Make']==Make)&(df_clean['PROFILE_Model']==Model)&\
                        (df_clean['PROFILE_Model_year']==ModelYear-2)]['MY_Average_' + item_small].reset_index(drop=True)[0]
            print('Success: ', df_clean['SPEC_MYMakeModel_gen'][i])
        except:
            df_clean['FLAG_Forecasted'][i] = False
            print('Failure')
df_clean = df_clean.drop(list(df_clean.filter(like='MY_Average').columns), axis=1)

965
2019_BMW_SERIES 3_7
Success:  2019_BMW_SERIES 3_7
966
2019_BMW_SERIES 3_7
Success:  2019_BMW_SERIES 3_7
967
2019_BMW_SERIES 3_7
Success:  2019_BMW_SERIES 3_7
968
2019_BMW_SERIES 3_7
Success:  2019_BMW_SERIES 3_7
969
2019_BMW_SERIES 3_7
Success:  2019_BMW_SERIES 3_7
1173
2021_BMW_SERIES 4_2
Success:  2021_BMW_SERIES 4_2
1409
2020_BMW_X3_3
Success:  2020_BMW_X3_3
1411
2020_BMW_X3_3
Success:  2020_BMW_X3_3
1414
2020_BMW_X3_3
Success:  2020_BMW_X3_3
1447
2019_BMW_X4_2
Success:  2019_BMW_X4_2
1525
2019_BMW_X5_4
Success:  2019_BMW_X5_4
1526
2019_BMW_X5_4
Success:  2019_BMW_X5_4
1610
2021_CADILLAC_ESCALADE_5
Success:  2021_CADILLAC_ESCALADE_5
1611
2021_CADILLAC_ESCALADE_5
Success:  2021_CADILLAC_ESCALADE_5
1612
2021_CADILLAC_ESCALADE_5
Success:  2021_CADILLAC_ESCALADE_5
1613
2021_CADILLAC_ESCALADE_5
Success:  2021_CADILLAC_ESCALADE_5
2766
2021_CHEVROLET_TAHOE_5
Success:  2021_CHEVROLET_TAHOE_5
2767
2021_CHEVROLET_TAHOE_5
Success:  2021_CHEVROLET_TAHOE_5
2768
2021_CHEVROLET_TAHOE_5
Success

In [ ]:
df_clean = df_clean.rename(columns={'SPEC_duration_months1007':'SPEC_Warranty_wholevehicle_duration_Months',
                                    'SPEC_duration_months1010':'SPEC_Warranty_powertrain_duration_Months',
                                    'SPEC_duration_months1013':'SPEC_Warranty_anticorrosion_duration_Months',
                                    'SPEC_duration_months1016':'SPEC_Warranty_paint_duration_Months',
                                    'SPEC_distance_km1008':'SPEC_Warranty_wholevehicle_distance_km',
                                    'SPEC_distance_km1011':'SPEC_Warranty_powertrain_distance_km',
                                    'SPEC_distance_km1014':'SPEC_Warranty_anticorrosion_distance_km',
                                    'SPEC_distance_km1017':'SPEC_Warranty_paint_distance_km'})

In [66]:
df_model_HighPrice = df_clean[df_clean['CATEGORY'] =='HighPrice'].drop('CATEGORY', axis=1)
df_model_MidPrice_Midsize = df_clean[df_clean['CATEGORY'] =='MidPrice_Midsize'].drop('CATEGORY', axis=1)
df_model_LowPrice_Smallsize = df_clean[df_clean['CATEGORY'] =='LowPrice_Smallsize'].drop('CATEGORY', axis=1)
df_model_MidPrice_Largesize = df_clean[df_clean['CATEGORY'] =='MidPrice_Largesize'].drop('CATEGORY', axis=1)

# Machine Learning Evaluation

## LowPrice Small size models

In [126]:
model_name = 'LowPrice_Smallsize'

In [127]:
target = 'TARGET_Weighted_RetailPrice'

In [128]:
features = pd.read_csv('../Modeling/Modeling_3rd_EVseparation/variables_union'+ model_name +'.csv')

In [129]:
df_test = df_model_LowPrice_Smallsize[df_model_LowPrice_Smallsize['FLAG_Forecasted']==True]

In [130]:
df_train = df_model_LowPrice_Smallsize[df_model_LowPrice_Smallsize['FLAG_Forecasted']==False]

In [131]:
test_y = df_test[[target]]

In [132]:
test_X = df_test[features['0'].str.replace('SPEC_distance_km1011', 'SPEC_Warranty_powertrain_distance_km').values]

In [133]:
train_X = df_train[features['0'].str.replace('SPEC_distance_km1011', 'SPEC_Warranty_powertrain_distance_km').values]

In [134]:
train_y = df_train[[target]]
train_y = np.log(train_y)

In [139]:
model = xgb.XGBRegressor(**{'random_state': 1, 'n_estimators': 500, 'max_depth': 3, 'learning_rate': 0.1, 'gamma': 0.0})

In [140]:
model.fit(train_X, train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.0, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=1, ...)

In [141]:
pred_y = model.predict(test_X)

In [142]:
rmse(np.exp(pred_y), test_y['TARGET_Weighted_RetailPrice'])

rmse 1015.6262577332475


1015.6262577332475

In [144]:
np.exp(pred_y)

array([27676.771, 28268.035, 29767.152, 30979.668, 36376.785, 25866.27 ,
       31329.754, 34973.207, 26660.719, 29306.047, 29301.66 , 28183.672,
       23499.86 , 24185.643, 24373.846, 24601.938, 29213.488, 30433.766,
       32832.926, 33277.723, 23775.871, 20268.098, 20052.438, 17944.863,
       15428.79 , 17904.06 , 14334.193, 15336.165, 24273.334, 30372.762,
       26176.59 , 29074.354, 26104.27 , 27436.676, 31989.14 , 33975.547,
       26888.662, 28646.754, 15603.917, 17542.951, 24058.932, 30173.266,
       26015.248, 27586.201, 21233.006, 25559.75 , 23071.258, 23606.71 ,
       23811.496, 27885.57 , 25601.59 , 28861.275, 27921.068, 32113.578,
       28607.305, 33905.37 , 34242.28 , 19526.17 , 19598.574, 21948.23 ,
       29029.193, 34673.12 , 24069.535, 25196.453, 32089.79 , 32827.32 ,
       27036.387, 27345.38 , 16377.189], dtype=float32)

In [145]:
test_y['TARGET_Weighted_RetailPrice']

3157     27099.109589
3158     28599.109589
3159     29259.109589
3160     30759.109589
3163     36715.000000
             ...     
11902    33575.089180
11903    34975.089180
11904    27375.089180
11905    28775.089180
12677    15499.315068
Name: TARGET_Weighted_RetailPrice, Length: 69, dtype: float64

## MidPrice_Largesize

In [183]:
model_name = 'MidPrice_Largesize'

In [184]:
target = 'TARGET_Weighted_RetailPrice'
model_gen = '3rd'

In [185]:
features = pd.read_csv('../01_Outputs/modeling_'+ model_gen + '/df_model_'+model_name+'.csv').dropna()

In [186]:
features = features.rename(columns={'SPEC_duration_months1007':'SPEC_Warranty_wholevehicle_duration_Months',
                                    'SPEC_duration_months1010':'SPEC_Warranty_powertrain_duration_Months',
                                    'SPEC_duration_months1013':'SPEC_Warranty_anticorrosion_duration_Months',
                                    'SPEC_duration_months1016':'SPEC_Warranty_paint_duration_Months',
                                    'SPEC_distance_km1008':'SPEC_Warranty_wholevehicle_distance_km',
                                    'SPEC_distance_km1011':'SPEC_Warranty_powertrain_distance_km',
                                    'SPEC_distance_km1014':'SPEC_Warranty_anticorrosion_distance_km',
                                    'SPEC_distance_km1017':'SPEC_Warranty_paint_distance_km'})

In [187]:
features = features.columns

In [188]:
df_test = df_model_LowPrice_Smallsize[df_model_LowPrice_Smallsize['FLAG_Forecasted']==True]

In [189]:
df_train = df_model_LowPrice_Smallsize[df_model_LowPrice_Smallsize['FLAG_Forecasted']==False]

In [190]:
test_y = df_test[[target]]

In [191]:
test_X = df_test[features].drop(target, axis=1)

In [192]:
train_X = df_train[features].drop(target, axis=1)

In [193]:
train_y = df_train[[target]]
train_y = np.log(train_y)

In [194]:
model = xgb.XGBRegressor(**{'random_state': 1, 'n_estimators': 500, 'max_depth': 7, 'learning_rate': 0.1, 'gamma': 0.0})

In [195]:
model.fit(train_X, train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.0, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=7, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=1, ...)

In [196]:
pred_y = model.predict(test_X)

In [197]:
rmse(np.exp(pred_y), test_y['TARGET_Weighted_RetailPrice'])

rmse 833.4500390031344


833.4500390031344

In [198]:
np.exp(pred_y)

array([27273.81 , 27283.02 , 30449.936, 30434.316, 37325.516, 25422.422,
       30755.531, 34818.957, 26377.064, 29019.754, 28974.54 , 27906.215,
       21799.537, 22735.22 , 23901.432, 24388.05 , 30287.143, 30143.037,
       32770.547, 33014.5  , 24127.52 , 20190.697, 20345.852, 17333.164,
       14887.231, 17426.479, 14696.695, 14864.392, 24478.975, 30536.072,
       26384.94 , 28351.053, 26698.885, 28113.527, 33167.97 , 34421.312,
       27257.74 , 28966.887, 15821.281, 17585.582, 22870.984, 30280.385,
       25699.145, 27909.861, 22009.709, 26041.684, 23029.424, 23367.291,
       24418.516, 28545.143, 25567.21 , 29714.398, 28834.314, 32544.252,
       29217.752, 33473.727, 34407.82 , 19293.145, 19581.928, 22531.826,
       30430.486, 35169.203, 26145.678, 27011.98 , 33137.742, 33984.2  ,
       27440.654, 28259.166, 15630.815], dtype=float32)

In [199]:
test_y['TARGET_Weighted_RetailPrice']

3157     27099.109589
3158     28599.109589
3159     29259.109589
3160     30759.109589
3163     36715.000000
             ...     
11902    33575.089180
11903    34975.089180
11904    27375.089180
11905    28775.089180
12677    15499.315068
Name: TARGET_Weighted_RetailPrice, Length: 69, dtype: float64

## MidPrice_Midsize

In [200]:
model_name = 'MidPrice_Midsize'

In [201]:
target = 'TARGET_Weighted_RetailPrice'
model_gen = '3rd'

In [202]:
features = pd.read_csv('../01_Outputs/modeling_'+ model_gen + '/df_model_'+model_name+'.csv').dropna()

In [203]:
features = features.rename(columns={'SPEC_duration_months1007':'SPEC_Warranty_wholevehicle_duration_Months',
                                    'SPEC_duration_months1010':'SPEC_Warranty_powertrain_duration_Months',
                                    'SPEC_duration_months1013':'SPEC_Warranty_anticorrosion_duration_Months',
                                    'SPEC_duration_months1016':'SPEC_Warranty_paint_duration_Months',
                                    'SPEC_distance_km1008':'SPEC_Warranty_wholevehicle_distance_km',
                                    'SPEC_distance_km1011':'SPEC_Warranty_powertrain_distance_km',
                                    'SPEC_distance_km1014':'SPEC_Warranty_anticorrosion_distance_km',
                                    'SPEC_distance_km1017':'SPEC_Warranty_paint_distance_km'})

In [204]:
features = features.columns

In [205]:
df_test = df_model_LowPrice_Smallsize[df_model_LowPrice_Smallsize['FLAG_Forecasted']==True]

In [206]:
df_train = df_model_LowPrice_Smallsize[df_model_LowPrice_Smallsize['FLAG_Forecasted']==False]

In [207]:
test_y = df_test[[target]]

In [208]:
test_X = df_test[features].drop(target, axis=1)

In [209]:
train_X = df_train[features].drop(target, axis=1)

In [210]:
train_y = df_train[[target]]
train_y = np.log(train_y)

In [211]:
model = xgb.XGBRegressor(**{'random_state': 1, 'n_estimators': 500, 'max_depth': 10, 'learning_rate': 0.1, 'gamma': 0.0})

In [212]:
model.fit(train_X, train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.0, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=1, ...)

In [213]:
pred_y = model.predict(test_X)

In [214]:
rmse(np.exp(pred_y), test_y['TARGET_Weighted_RetailPrice'])

rmse 856.3473873435446


856.3473873435446

In [215]:
np.exp(pred_y)

array([27157.531, 28332.89 , 28241.762, 30131.254, 35325.44 , 25690.445,
       30888.098, 33928.63 , 25456.266, 28685.932, 28216.457, 28006.38 ,
       21043.572, 22560.059, 23344.617, 23636.043, 30025.234, 31461.045,
       32337.771, 33146.402, 25803.098, 19496.547, 20617.275, 18412.613,
       15325.9  , 18386.627, 14447.21 , 15215.236, 24035.287, 30056.75 ,
       25557.824, 26977.637, 26293.857, 26951.1  , 30151.29 , 32884.098,
       27324.473, 28894.82 , 16378.985, 17243.67 , 22873.078, 30003.195,
       24911.932, 27008.09 , 21671.064, 25881.84 , 22567.44 , 23285.184,
       24262.387, 28341.998, 25607.133, 30145.51 , 28190.957, 32536.62 ,
       28989.383, 33545.598, 34201.453, 19006.117, 19456.816, 22516.81 ,
       29997.387, 34802.754, 25192.418, 26506.148, 32111.74 , 34601.277,
       27257.947, 28310.77 , 15882.978], dtype=float32)

In [216]:
test_y['TARGET_Weighted_RetailPrice']

3157     27099.109589
3158     28599.109589
3159     29259.109589
3160     30759.109589
3163     36715.000000
             ...     
11902    33575.089180
11903    34975.089180
11904    27375.089180
11905    28775.089180
12677    15499.315068
Name: TARGET_Weighted_RetailPrice, Length: 69, dtype: float64

In [220]:
XGB_importances = model.feature_importances_
XGB_importances = pd.DataFrame(XGB_importances, columns=['importances'], index=test_X.columns)

In [224]:
len(test_X.columns)

232